## Contexto do problema

Instituições financeiras liram diariamente com milhares de operações internas e transações que, quando mal monitoradas, podem gerar **risco operacional**.

Risco operacional envolve falhas em processos, sistemas, pessoas ou eventos externos, podendo resultar em:
- perdas financeiras
- falhas de compliance 
- impactos reputacionais

## Cenário simulado 

Neste projeto, será simulada uma base de dados contendo registros operacionais de uma instituição financeira fictícia 

Cada registro representa uma ocorrência operacional associada a:
- área responsável
- tipo de evento 
- data
- valor financeiro impactado 
- canal de origem 

O objetivo é identificar padrões que possam indicar **aumento de risco operacional**.

## Estrutura dos Dados

Cada linha do dataset representa uma ocorrência de risco operacional registrada pela instituição

A seguir estão descritas as principais variáveis consideradas na análise.

## Variáveis do Dataset 

- **id_ocorrencia**: identificador único da ocorrência
- **data_ocorrencia**: data em que o evento foi registrado 
- **area responsavel**: área interna responsável pelo processo
- **tipo_evento**: classificação do evento operacional 
- **canal_origem**: canal onde o evento ocorreu (ex: digital, agência, backoffice)
- **valor_impacto**: valor financeiro estimado do impacto
- **nivel_risco**: classificação do risco (baixo, médio, alto)